In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn
import datetime as dt
%matplotlib inline 

In [3]:
df = pd.read_csv('My Uber Drives - 2016.csv')

In [4]:
df.head()

,START_DATE*,END_DATE*,CATEGORY*,START*,STOP*,MILES*,PURPOSE*
0,1/1/2016 21:11,1/1/2016 21:17,Business,Fort Pierce,Fort Pierce,5.1,Meal/Entertain
1,1/2/2016 1:25,1/2/2016 1:37,Business,Fort Pierce,Fort Pierce,5.0,NaN
2,1/2/2016 20:25,1/2/2016 20:38,Business,Fort Pierce,Fort Pierce,4.8,Errand/Supplies
3,1/5/2016 17:31,1/5/2016 17:45,Business,Fort Pierce,Fort Pierce,4.7,Meeting
4,1/6/2016 14:42,1/6/2016 15:49,Business,Fort Pierce,West Palm Beach,63.7,Customer Visit


In [5]:
df.size

8092

In [6]:
df.shape

(1156, 7)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1156 entries, 0 to 1155
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   START_DATE*  1156 non-null   object 
 1   END_DATE*    1155 non-null   object 
 2   CATEGORY*    1155 non-null   object 
 3   START*       1155 non-null   object 
 4   STOP*        1155 non-null   object 
 5   MILES*       1156 non-null   float64
 6   PURPOSE*     653 non-null    object 
dtypes: float64(1), object(6)
memory usage: 63.3+ KB


In [8]:
df.describe()

,MILES*
count,1156.000000
mean,21.115398
std,359.299007
min,0.500000
25%,2.900000
50%,6.000000
75%,10.400000
max,12204.700000


In [9]:
df.columns

Index(['START_DATE*', 'END_DATE*', 'CATEGORY*', 'START*', 'STOP*', 'MILES*',
       'PURPOSE*'],
      dtype='object')

In [10]:
# Rename columns
df.rename(columns = {'START_DATE*' : 'StartDate', 'END_DATE*' : 'EndDate', 'CATEGORY*' : 'Category', 'START*': 'Start', 'STOP*' : 'Stop',
 'MILES*' : 'Miles', 'PURPOSE*' : 'Purpose'}, inplace = True )


In [11]:
df.columns

Index(['StartDate', 'EndDate', 'Category', 'Start', 'Stop', 'Miles',
       'Purpose'],
      dtype='object')

In [12]:
# Remove last row from table
df.iloc[[1155]]

,StartDate,EndDate,Category,Start,Stop,Miles,Purpose
1155,Totals,NaN,NaN,NaN,NaN,12204.7,NaN


In [13]:
df.drop([1155], axis = 0, inplace = True)

In [14]:
df.shape

(1155, 7)

In [15]:
df.dtypes

StartDate     object
EndDate       object
Category      object
Start         object
Stop          object
Miles        float64
Purpose       object
dtype: object

In [16]:
# Convert datatype of StartDate into datetime
df.StartDate = pd.to_datetime(df.StartDate)

In [17]:
df.EndDate = pd.to_datetime(df.EndDate)

In [18]:
df.dtypes

StartDate    datetime64[ns]
EndDate      datetime64[ns]
Category             object
Start                object
Stop                 object
Miles               float64
Purpose              object
dtype: object

<h4>* Datatypes in suitable format

In [19]:
# chech null
df.isnull().sum()

StartDate      0
EndDate        0
Category       0
Start          0
Stop           0
Miles          0
Purpose      502
dtype: int64

In [20]:
na = 'Not Applicable'
df.Purpose = df.Purpose.fillna('NA')
df.isnull().sum()

StartDate    0
EndDate      0
Category     0
Start        0
Stop         0
Miles        0
Purpose      0
dtype: int64

In [21]:
df.Purpose.head(20)

0      Meal/Entertain
1                  NA
2     Errand/Supplies
3             Meeting
4      Customer Visit
5      Meal/Entertain
6             Meeting
7             Meeting
8             Meeting
9      Customer Visit
10            Meeting
11            Meeting
12            Meeting
13     Temporary Site
14    Errand/Supplies
15     Meal/Entertain
16     Meal/Entertain
17     Meal/Entertain
18     Meal/Entertain
19    Errand/Supplies
Name: Purpose, dtype: object

In [22]:
# check duplicates
df.duplicated().sum()

1

In [23]:
# remove duplicate entry
df = df.drop_duplicates()
df.duplicated().sum()

0

In [24]:
df['Year'] = df.StartDate.dt.year
df['Year'].head()

0    2016
1    2016
2    2016
3    2016
4    2016
Name: Year, dtype: int64

In [25]:
df['Month'] = df.StartDate.dt.month
df['Month'].head()

0    1
1    1
2    1
3    1
4    1
Name: Month, dtype: int64

In [26]:
df['Day'] = df.StartDate.dt.day
df['Day'].head()

0    1
1    2
2    2
3    5
4    6
Name: Day, dtype: int64

In [27]:
df['StartHour'] = df.StartDate.dt.hour
df['StartHour'].head()

0    21
1     1
2    20
3    17
4    14
Name: StartHour, dtype: int64

In [28]:
def categorize_hour(hour):
    if hour >= 0 and hour < 6:
        return 'Night'
    elif hour >= 6 and hour < 18:
        return 'Day'
    else:
        return 'Night'

In [29]:
df['Time of Day'] = df['StartHour'].apply(categorize_hour)
df['Time of Day'].head()

0    Night
1    Night
2    Night
3      Day
4      Day
Name: Time of Day, dtype: object

In [30]:
df['Time Difference'] = df['EndDate'] - df['StartDate']
df['Time Difference'].head()

0   0 days 00:06:00
1   0 days 00:12:00
2   0 days 00:13:00
3   0 days 00:14:00
4   0 days 01:07:00
Name: Time Difference, dtype: timedelta64[ns]

In [31]:
df['Total Hours'] = (df['Time Difference'].apply(lambda x: x.total_seconds() / 3600)).round(2)
df['Total Hours'].head()

0    0.10
1    0.20
2    0.22
3    0.23
4    1.12
Name: Total Hours, dtype: float64

In [32]:
df['MoreThan1Hour'] = df['Total Hours'].apply(lambda x : x > 1, True, False)
df['MoreThan1Hour'].head()

0    False
1    False
2    False
3    False
4     True
Name: MoreThan1Hour, dtype: bool

In [33]:
df['Time Difference (Minutes)'] = ((df['EndDate'] - df['StartDate']).dt.total_seconds() / 60).round(2)
df['Time Difference (Minutes)'].head()

0     6.0
1    12.0
2    13.0
3    14.0
4    67.0
Name: Time Difference (Minutes), dtype: float64

In [34]:
df.head()

,StartDate,EndDate,Category,Start,Stop,Miles,Purpose,Year,Month,Day,StartHour,Time of Day,Time Difference,Total Hours,MoreThan1Hour,Time Difference (Minutes)
0,2016-01-01 21:11:00,2016-01-01 21:17:00,Business,Fort Pierce,Fort Pierce,5.1,Meal/Entertain,2016,1,1,21,Night,0 days 00:06:00,0.10,False,6.0
1,2016-01-02 01:25:00,2016-01-02 01:37:00,Business,Fort Pierce,Fort Pierce,5.0,NA,2016,1,2,1,Night,0 days 00:12:00,0.20,False,12.0
2,2016-01-02 20:25:00,2016-01-02 20:38:00,Business,Fort Pierce,Fort Pierce,4.8,Errand/Supplies,2016,1,2,20,Night,0 days 00:13:00,0.22,False,13.0
3,2016-01-05 17:31:00,2016-01-05 17:45:00,Business,Fort Pierce,Fort Pierce,4.7,Meeting,2016,1,5,17,Day,0 days 00:14:00,0.23,False,14.0
4,2016-01-06 14:42:00,2016-01-06 15:49:00,Business,Fort Pierce,West Palm Beach,63.7,Customer Visit,2016,1,6,14,Day,0 days 01:07:00,1.12,True,67.0


In [35]:
df['month_name'] = df['StartDate'].dt.month_name()
df['month_name'].head()

0    January
1    January
2    January
3    January
4    January
Name: month_name, dtype: object

In [36]:
df['DayName'] = df['StartDate'].dt.day_name()
df['DayName'].head()

0       Friday
1     Saturday
2     Saturday
3      Tuesday
4    Wednesday
Name: DayName, dtype: object

In [270]:
df.to_csv('UberCleanDate.csv')